In [1]:
print("hello")

hello


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# Check for GPU availability (MPS for Apple Silicon or CUDA for NVIDIA GPUs)
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the ESD-X model from Hugging Face
model_id = "kevinlu4588/esd-x"

try:
    pipeline = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device != "cpu" else torch.float32,  # Use FP16 for GPU, FP32 for CPU
        use_safetensors=True
    ).to(device)

    print("✅ ESD-X model loaded successfully!")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# Sample text prompt for generation
prompt = "A serene landscape with mountains and a clear blue sky"

# Inference
with torch.inference_mode():
    image = pipeline(prompt).images[0]

# Display the generated image
image.show()

# Optionally save the image
image.save("output_image.png")
